### Bow 기반의 문서 분류

주어진 문서를 미리 정의된 클래스로 분류하는 작업

<머신러닝의 분류 작업>

1. 로지스틱 회귀분석
2. 결정트리
3. 나이브 베이즈 **(가장 많이 사용)**
4. SVM

※ 주의사항

뉴스의 분야가 라벨로 달려 있어야 한다.

In [4]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroup_train = fetch_20newsgroups(subset = 'train', remove = ('headers', 'footers',
                                                                 'quotes'),
                                     categories = categories)
newsgroup_test = fetch_20newsgroups(subset = 'test', remove = ('headers', 'footers',
                                                                 'quotes'),
                                     categories = categories)

print('#Train set size : ', len(newsgroup_train.data))
print('#Test set size : ', len(newsgroup_test.data))
print('#Selected categories : ', newsgroup_train.target_names)
print('Train labels : ', set(newsgroup_train.target))

#Train set size :  2034
#Test set size :  1353
#Selected categories :  ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
Train labels :  {0, 1, 2, 3}


In [5]:
print("#Train set text samples : ", newsgroup_train.data[0])
print("#Train set label samples : ", newsgroup_train.target[0])
print("#Test set text samples : ", newsgroup_test.data[0])
print("#Test set label samples : ", newsgroup_test.target[0])

#Train set text samples :  Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
#Train set label samples :  1
#Test set text samples :  TRry the SKywatch project in  Arizona.
#Test set label samples :  2


#### 카운트 기반 특성 추출

In [6]:
X_train = newsgroup_train.data
y_train = newsgroup_train.target

X_test = newsgroup_test.data
y_test = newsgroup_test.target

from sklearn.feature_extraction.text import CountVectorizer

# min_df : 단어가 최소 이 문서에 5개는 나타나야 한다. (문서에서 거의 쓰이지 않는다 - 예외)
# max_df : 단어가 문서의 50%를 초과해 나타나는 단어를 제외한다.
cv = CountVectorizer(max_features = 2000, min_df = 5, max_df = 0.5)

X_train_cv = cv.fit_transform(X_train)
print('Train set dimension : ', X_train_cv.shape) # (train 문서의 수, 특성의 개수)
X_test_cv = cv.transform(X_test)
print('Test set dimension', X_test_cv.shape) # (test 문서의 수, 특성의 개수)

Train set dimension :  (2034, 2000)
Test set dimension (1353, 2000)


In [7]:
for word, count in zip(
    # 사용된 단어의 이름과 빈도 수 추출
    cv.get_feature_names_out()[:100], X_train_cv[0].toarray()[0, :100]
):
    print(word, ':', count, end = ', ')

00 : 0, 000 : 0, 01 : 0, 04 : 0, 05 : 0, 10 : 0, 100 : 0, 1000 : 0, 11 : 0, 12 : 0, 128 : 0, 129 : 0, 13 : 0, 130 : 0, 14 : 0, 15 : 0, 16 : 0, 17 : 0, 18 : 0, 19 : 0, 1987 : 0, 1988 : 0, 1989 : 0, 1990 : 0, 1991 : 0, 1992 : 0, 1993 : 0, 20 : 0, 200 : 0, 202 : 0, 21 : 0, 22 : 0, 23 : 0, 24 : 0, 25 : 0, 256 : 0, 26 : 0, 27 : 0, 28 : 0, 2d : 0, 30 : 0, 300 : 0, 31 : 0, 32 : 0, 33 : 0, 34 : 0, 35 : 0, 39 : 0, 3d : 0, 40 : 0, 400 : 0, 42 : 0, 45 : 0, 50 : 0, 500 : 0, 60 : 0, 600 : 0, 65 : 0, 70 : 0, 75 : 0, 80 : 0, 800 : 0, 90 : 0, 900 : 0, 91 : 0, 92 : 0, 93 : 0, 95 : 0, _the : 0, ability : 0, able : 1, abortion : 0, about : 1, above : 0, absolute : 0, absolutely : 0, ac : 0, accept : 0, acceptable : 0, accepted : 0, access : 0, according : 0, account : 0, accurate : 0, across : 0, act : 0, action : 0, actions : 0, active : 0, activities : 0, activity : 0, acts : 0, actual : 0, actually : 0, ad : 0, add : 0, added : 0, addition : 0, additional : 0, address : 0, 

#### 1. 나이브베이즈를 이용한 문서 분류

새로운 기사에 나온 단어들이 각각 어떤 확률로 경제 관련 기사와 과학 기사에 나오는지 계산하고, 이를 잘 결합하여
**새로운 기사가 어느 분야**에 속하는 지 확인한다.

사전확률을 계산하여, 특성을 고려해 이를 더 정확한 확률로 바꾼다.

Q. MultinomialNB
이산적인 특성 값들을 이용해 분류하고자 할 때 사용한다.
이산적이란, 연속적인 값이 아닌 값이라는 뜻으로 countvector가 이에 해당한다.

매개변수
alpha : 모델의 복잡도 조절 (이 값을 늘리면, 통계 데이터가 완만해지고 복잡도가 낮아진다.)
TFidfVectorizer 사용 가능

In [8]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB()
NB_clf.fit(X_train_cv, y_train)

print('Train set score : {:.3f}'.format(NB_clf.score(X_train_cv, y_train)))
print('Test set score : {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))

Train set score : 0.824
Test set score : 0.732


In [11]:
print('#First document and label in test data : ', X_test[0], y_test[0])
print('#second document and label in test data : ', X_test[1], y_test[1])

pred = NB_clf.predict(X_test_cv[:2])

print('#Predicted Labels : ', pred)

# 주제 (topic) 전달
print(
    '#Predicted categories : ',
    newsgroup_train.target_names[pred[0]],
    newsgroup_train.target_names[pred[1]]
)

#First document and label in test data :  TRry the SKywatch project in  Arizona. 2
#second document and label in test data :  The Vatican library recently made a tour of the US.
 Can anyone help me in finding a FTP site where this collection is 
 available. 1
#Predicted Labels :  [2 1]
#Predicted categories :  sci.space comp.graphics


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=2000, min_df = 5, max_df = 0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

NB_clf.fit(X_train_tfidf, y_train)

print('Train set score : {:.3f}'. format(NB_clf.score(X_train_tfidf, y_train)))
print('Test set score : {:.3f}'. format(NB_clf.score(X_test_tfidf, y_test)))

Train set score : 0.862
Test set score : 0.741


In [19]:
hasattr(NB_clf, 'coef_')

False

In [21]:
import numpy as np

def top10_features(classifier, vectorizer, categories):
    # word 이름 추출
    feature_names = np.asarray(vectorizer.get_feature_names_out())
    for i, category in enumerate(categories):
        # 역순 정렬을 위해 계수에 음수를 취해 정렬 후, 앞에서 10개 추출
        top10 = np.argsort(-classifier.feature_log_prob_[i])[:10]
        # 카테고리와 영향이 큰 특성 10개 반환
        print("%s : %s" % (category, ",".join(feature_names[top10])))

top10_features(NB_clf, tfidf, newsgroup_train.target_names)

alt.atheism : you,not,are,be,this,have,as,what,they,if
comp.graphics : you,on,graphics,this,have,any,can,or,with,thanks
sci.space : space,on,you,be,was,this,as,they,have,are
talk.religion.misc : you,not,he,are,as,this,be,god,was,they


#### 로지스틱 회귀분석을 이용한 문서 분류

회귀분석은 예측하고자 하는 값이 연속적일 때 사용하는 반면, 로지스틱 회귀 분석은 라벨이 연속적인 값이 아니고 분류에 해당될 때 사용한다.


In [22]:
from sklearn.linear_model import LogisticRegression

LR_clf = LogisticRegression()

LR_clf.fit(X_train_tfidf, y_train)

print('Train set score : {:.3f}'. format(LR_clf.score(X_train_tfidf, y_train)))
print('Test set score : {:.3f}'. format(LR_clf.score(X_test_tfidf, y_test))) # 과적합 발생 가능성 있음

Train set score : 0.930
Test set score : 0.734


#### 과적합 해결 방법

※ 릿지 회귀를 이용한 과적합 방지

릿지 회귀는 회귀분석에 정규화를 사용하는 알고리즘으로, 최적화를 위한 목적함수에 정규화 항목을 넣어서 특성에 대한 계수가 지나치게 커지는 것을 억제한다.

Ridge Classfier의 매개변수
alpha : 정규화의 정도 조절 (값이 커질 수록, 정규화의 비중이 커져 계수를 더 많이 억제한다.)


In [23]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier()

ridge_clf.fit(X_train_tfidf, y_train)

print('Train set score : {:.3f}'. format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score : {:.3f}'. format(ridge_clf.score(X_test_tfidf, y_test))) # 과적합 발생 가능성 있음

Train set score : 0.960
Test set score : 0.735


#### 과적합 해결방법

1 . validation set을 이용한 그리드 서치
2 . 하이퍼파라미터 튜닝

In [26]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

max_score = 0
max_alpha = 0

for alpha in np.arange(0.1, 10, 0.1):
    ridge_clf = RidgeClassifier(alpha = alpha)
    ridge_clf.fit(X_train_ridge, y_train_ridge)
    score = ridge_clf.score(X_val_ridge, y_val_ridge)
    if score > max_score :
        max_score = score
        max_alpha = alpha

print('Max Alpha {:3f} at max validation score {:3f}'.format(max_alpha, max_score))

Max Alpha 1.600000 at max validation score 0.825553


In [28]:
ridge_clf = RidgeClassifier(alpha = 1.6)
ridge_clf.fit(X_train_tfidf, y_train)

print('Train set score : {:.3f}'. format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score : {:.3f}'. format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score : 0.947
Test set score : 0.739


In [32]:
import numpy as np

def top10_features(classifier, vectorizer, categories):
    # word 이름 추출
    feature_names = np.asarray(vectorizer.get_feature_names_out())
    for i, category in enumerate(categories):
        # 역순 정렬을 위해 계수에 음수를 취해 정렬 후, 앞에서 10개 추출
        top10 = np.argsort(-classifier.coef_[i])[:10]
        # 카테고리와 영향이 큰 특성 10개 반환
        print("%s : %s" % (category, ",".join(feature_names[top10])))

In [33]:
top10_features(ridge_clf, tfidf, newsgroup_train.target_names)

alt.atheism : bobby,religion,atheism,atheists,motto,punishment,islam,deletion,islamic,satan
comp.graphics : graphics,computer,3d,file,image,hi,42,using,screen,looking
sci.space : space,orbit,nasa,spacecraft,moon,sci,launch,flight,funding,idea
talk.religion.misc : christian,christians,fbi,blood,order,jesus,objective,children,christ,hudson


#### 라쏘 회귀를 이용한 특성 선택

특성의 계수에 대해 정규화를 한다.
라쏘는 정규화를 할 때, 특성의 계수가 0에 가까워지면, 이를 완전히 0으로 바꾼다. (절댓값)
어떤 특성의 계수가 0이라는 것은 그 특성은 분류에 전혀 영향을 미치지 않는다는 것이다.
-> 특성의 수를 줄여주는 효과가 있다. (정확도가 정확히 향상된다고 보기는 어렵다.)

In [34]:
lasso_clf = LogisticRegression(penalty='l1', solver = 'liblinear', C = 1) # C는 alpha의 역수

lasso_clf.fit(X_train_tfidf, y_train)

print('Train set score : {:.3f}'. format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score : {:.3f}'. format(lasso_clf.score(X_test_tfidf, y_test)))

Train set score : 0.819
Test set score : 0.724


In [35]:
print(
    '#Used features count : {}'.format(np.sum(lasso_clf.coef_ != 0)), #특성 선택
    'out of',
    X_train_tfidf.shape[1]
)

#Used features count : 437 out of 2000


In [36]:
top10_features(lasso_clf, tfidf, newsgroup_train.target_names)

alt.atheism : bobby,atheism,atheists,islam,religion,islamic,motto,atheist,satan,vice
comp.graphics : graphics,image,3d,file,computer,hi,video,files,looking,sphere
sci.space : space,orbit,launch,nasa,spacecraft,flight,moon,dc,shuttle,solar
talk.religion.misc : fbi,christian,christians,christ,order,jesus,children,objective,context,blood


#### 결정트리를 이용한 문서 분류 방법

1. DecisionTree
2. RandomForest
3. GradientBoosting

결정트리가 과적합되는 성향이 매우 강하다.

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

tree = DecisionTreeClassifier(random_state=7)
tree.fit(X_train_tfidf, y_train)
print(
    'Train set score : {:.3f}'. format(tree.score(X_train_tfidf, y_train)),
    'Test set score : {:.3f}'. format(tree.score(X_test_tfidf, y_test))
)

forest = RandomForestClassifier(random_state=7)
forest.fit(X_train_tfidf, y_train)
print(
    'Train set score : {:.3f}'. format(forest.score(X_train_tfidf, y_train)),
    'Test set score : {:.3f}'. format(forest.score(X_test_tfidf, y_test))
)

gb = GradientBoostingClassifier(random_state=7)
gb.fit(X_train_tfidf, y_train)
print(
    'Train set score : {:.3f}'. format(gb.score(X_train_tfidf, y_train)),
    'Test set score : {:.3f}'. format(gb.score(X_test_tfidf, y_test))
)

Train set score : 0.977 Test set score : 0.536
Train set score : 0.977 Test set score : 0.685
Train set score : 0.933 Test set score : 0.696


In [38]:
sorted_feature_importances = sorted(
    zip(tfidf.get_feature_names_out(), gb.feature_importances_),
    key = lambda x:x[1],
    reverse=True
)

for feature, value in sorted_feature_importances[:40]:
    print('%s : %.3f' % (feature, value), end = ', ')

space : 0.126, graphics : 0.080, atheism : 0.024, thanks : 0.023, file : 0.021, orbit : 0.020, jesus : 0.018, god : 0.018, hi : 0.017, nasa : 0.015, image : 0.015, files : 0.014, christ : 0.010, moon : 0.010, bobby : 0.010, launch : 0.010, looking : 0.010, christian : 0.010, atheists : 0.009, christians : 0.009, fbi : 0.009, 3d : 0.008, you : 0.008, not : 0.008, islamic : 0.007, religion : 0.007, spacecraft : 0.007, flight : 0.007, computer : 0.007, islam : 0.007, ftp : 0.006, color : 0.006, software : 0.005, atheist : 0.005, card : 0.005, people : 0.005, koresh : 0.005, his : 0.005, kent : 0.004, sphere : 0.004, 

#### 성능 높이기

In [42]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words('english')

from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import re

RegTok = RegexpTokenizer("[\w']{3,}")
english_stops = set(stopwords.words('english'))

def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    words = [word for word in tokens if (word not in english_stops) and len(word) > 2]
    features = (list(map(lambda token: PorterStemmer().stem(token), words)))
    return features

tfidf = TfidfVectorizer(tokenizer = tokenizer, max_features= 2000, max_df = 0.5, min_df = 5)

X_train_tfidf= tfidf.fit_transform(X_train)
X_test_tfidf=  tfidf.transform(X_test)

LR_clf = LogisticRegression()
LR_clf.fit(X_train_tfidf, y_train)

print('Train set score : {:.3f}'. format(LR_clf.score(X_train_tfidf, y_train)))
print('Test set score : {:.3f}'. format(LR_clf.score(X_test_tfidf, y_test)))

Train set score : 0.930
Test set score : 0.751


In [44]:
# 특성 추출기의 단어 제한을 없앤다
tfidf = TfidfVectorizer(tokenizer = tokenizer, max_df = 0.5, min_df = 5)

X_train_tfidf= tfidf.fit_transform(X_train)
X_test_tfidf=  tfidf.transform(X_test)

LR_clf = LogisticRegression()
LR_clf.fit(X_train_tfidf, y_train)

print("#Train set Dimension : ", X_train_tfidf.shape)
print('Train set score : {:.3f}'. format(LR_clf.score(X_train_tfidf, y_train)))
print('Test set score : {:.3f}'. format(LR_clf.score(X_test_tfidf, y_test)))

C:\Users\ahyeo\anaconda3\envs\TextMining\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


#Train set Dimension :  (2034, 4056)
Train set score : 0.953
Test set score : 0.761


### 카운트 기반의 문제점과 N-gram을 이용한 보완